In [12]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/jwfree/anaconda3/envs/CLASSIFIER/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [13]:
import numpy as np
from sklearn import metrics
import os

In [14]:
os.listdir("/home/jwfree/rc0_data")

['curvature', 'angle', 'direction', 'area']

In [15]:
import copy
import statistics as s
results_template = {
    "vgg19": {
        "mlae": [],
        "msqr_error": [],
        "total_good_results": [],
        "total_bad_results": []
    },
    "alexnet": {
        "mlae": [],
        "msqr_error": [],
        "total_good_results": [],
        "total_bad_results": []
    },
    "densenet": {
        "mlae": [],
        "msqr_error": [],
        "total_good_results": [],
        "total_bad_results": []
    },
    "resnet": {
        "mlae": [],
        "msqr_error": [],
        "total_good_results": [],
        "total_bad_results": []
    }
}
results = {}
results_total = {}
rc0 = "/home/jwfree/rc0_data"
rc1 = "/home/jwfree/rc1_data"
for cluster in [rc0, rc1]:
    for stim in os.listdir(cluster):
        
        results_total[stim] = {}
        
        results[stim] = copy.deepcopy(results_template)
        stim_path = os.path.join(cluster, stim)

        for model in os.listdir(stim_path):

            model_path = os.path.join(stim_path, model)
            results_file = os.path.join(model_path, "results.npz")

            if os.path.exists(results_file):

                files = os.listdir(model_path)
                key = None
                if "vgg19" in files:
                    key = "vgg19"
                elif "alexnet" in files:
                    key = "alexnet"
                elif "densenet" in files:
                    key = "densenet"
                elif "resnet" in files:
                    key = "resnet"
                else:
                    continue

                data = np.load(results_file)
                

                
                y_test  = data["labels"]
                l_min = y_test.min()
                l_max = y_test.max()
                
                y_test  = y_test / (l_max - l_min)
                
                y_pred = data["results"]
                y_pred = y_pred / (l_max - l_min)
                
                mlae = np.log2(metrics.mean_absolute_error(y_pred*100, y_test*100)+.125)
                
                msqr_error = np.sqrt(metrics.mean_squared_error(y_pred*100, y_test*100))

                results[stim][key]["mlae"].append(mlae.tolist())
                results[stim][key]["msqr_error"].append(msqr_error.tolist())
                results[stim][key]["total_bad_results"].append(len(data["bad_result_ids"]))
                results[stim][key]["total_good_results"].append(len(data["results"]))

        for model in results_template.keys():
            results_total[stim][model] = {}
            results_total[stim][model]["runs"] = len(results[stim][model]["msqr_error"])
            if results_total[stim][model]["runs"] != 0:
                results_total[stim][model]["mlae"] = results[stim][model]["mlae"]
                results_total[stim][model]["msqr_error"] = results[stim][model]["msqr_error"]
                results_total[stim][model]["total_bad_results"] = results[stim][model]["total_bad_results"]
                results_total[stim][model]["total_good_results"] = results[stim][model]["total_good_results"]
    
            
            

In [16]:
import yaml
print(yaml.dump(results_total))
with open("results.yml", "w") as file:
    yaml.dump(results_total, file)

angle:
  alexnet:
    mlae:
    - 2.1491938462653457
    - 2.1978265201172005
    - 2.0733010025223924
    - 2.155270662891321
    - 2.316610290747225
    - 2.0679348460322067
    msqr_error:
    - 5.950784569722479
    - 6.395232772409649
    - 5.70220594158449
    - 6.069457790836065
    - 6.899128619161804
    - 5.676331648575431
    runs: 6
    total_bad_results:
    - 5
    - 282
    - 96
    - 1
    - 33
    - 2
    total_good_results:
    - 19995
    - 19718
    - 19904
    - 19999
    - 19967
    - 19998
  densenet:
    mlae:
    - 2.2457269720363966
    - 2.2078850502464515
    - 2.2268530846075945
    - 2.3427661781763374
    - 2.3582902009095554
    - 2.516441782246769
    msqr_error:
    - 6.497060270050162
    - 6.312987563092664
    - 6.410123184082133
    - 7.038171153485496
    - 7.027421314738477
    - 7.76132794188536
    runs: 6
    total_bad_results:
    - 0
    - 65
    - 0
    - 21
    - 43
    - 17
    total_good_results:
    - 20000
    - 19935
    - 20000
    -